# Lab 5 (Part II): Ingesting Streaming Data with Kinesis
### MACS 30123: Large-Scale Computing for the Social Sciences (Spring 2020)

In this second part of the lab, we'll explore how we can use Kinesis to ingest streaming text data, of the sort we might encounter on Twitter.

To avoid requiring you to set up Twitter API access, we will create Twitter-like text and metadata using the `testdata` package to perform this demonstration. It should be easy enough to plug your streaming Twitter feed into this workflow if you desire to do so as an individual exercise (for instance, as a part of a final project!). Additionally, once you have this pipeline running, you can scale it up even further to include many more producers and consumers, if you would like, as discussed in lecture and the readings.

Recall from the lecture and readings that in a Kinesis workflow, "producers" send data into a Kinesis stream and "consumers" draw data out of that stream to perform operations on it (i.e. real-time processing, archiving raw data, etc.). To make this a bit more concrete, we are going to implement a simplified version of this workflow in this lab, in which we spin up Producer and Consumer (t2.nano) EC2 Instances and create a Kinesis stream. Our Producer instance will run a producer script (which writes our Twitter-like text data into a Kinesis stream) and our Consumer instance will run a consumer script (which reads the Twitter-like data and calculates a simple demo statistic -- the average unique word count per tweet, as a real-time running average).

You can visualize this data pipeline, like so:

<img src="simple_kinesis_architecture.png" width="800" align="left" />


To begin implementing this pipeline, let's import `boto3` and initialize the AWS services we'll be using in this lab (EC2 and Kinesis).

In [1]:
import boto3
import time

session = boto3.Session()

kinesis = session.client('kinesis')
ec2 = session.resource('ec2')
ec2_client = session.client('ec2')

Then, we need to create the Kinesis stream that our Producer EC2 instance will write streaming tweets to. Because we're only setting this up to handle traffic from one consumer and one producer, we'll just use one shard, but we could increase our throughput capacity by increasing the ShardCount if we wanted to do so.

In [2]:
response = kinesis.create_stream(StreamName = 'test_stream',
                                 ShardCount = 1
                                )

# Is the stream active and ready to be written to/read from? Wait until it exists before moving on:
waiter = kinesis.get_waiter('stream_exists')
waiter.wait(StreamName='test_stream')

OK, now we're ready to set up our producer and consumer EC2 instances that will write to and read from this Kinesis stream. Let's spin up our two EC2 instances (specified by the `MaxCount` parameter) using one of the Amazon Linux AMIs.  Notice here that you will need to specify your `.pem` file for the `KeyName` parameter, as well as create a custom security group/group ID. Designating a security group is necessary because, by default, AWS does not allow inbound ssh traffic into EC2 instances (they create custom ssh-friendly security groups each time you run the GUI wizard in the console). Thus, if you don't set this parameter, you will not be able to ssh into the EC2 instances that you create here with `boto3`. You can follow along in the lab video for further instructions on how you can set up one of these security groups.

Also, we need to specify an IAM Instance Profile so that our EC2 instances will have the permissions necessary to interact with other AWS services on our behalf. Here, I'm using one of the profiles we create in Part I of Lab 5 (a default AWS profile for launching EC2 instances within an EMR cluster), as this gives us all of the necessary permissions

In [3]:
instances = ec2.create_instances(ImageId='ami-0915e09cc7ceee3ab',
                                 MinCount=1,
                                 MaxCount=2,
                                 InstanceType='t2.micro',
                                 KeyName='MACS_30123',
                                 SecurityGroupIds=['sg-02248fb2c9eac8bef'],
                                 SecurityGroups=['MACS_30123'],
                                 IamInstanceProfile=
                                     {'Name': 'EMR_EC2_DefaultRole'},
                                )

# Wait until EC2 instances are running before moving on
waiter = ec2_client.get_waiter('instance_running')
waiter.wait(InstanceIds=[instance.id for instance in instances])

While we wait for these instances to start running, let's set up the Python scripts that we want to run on each instance. First of all, we have to define a script for our Producer instance, which continuously produces Twitter-like data using the `testdata` package and puts that data into our Kinesis stream.

In [4]:
%%file producer.py

import boto3
import testdata
import json

kinesis = boto3.client('kinesis', region_name='us-east-1')

# Continously write Twitter-like data into Kinesis stream
while 1 == 1:
    test_tweet = {'username': testdata.get_username(),
                  'tweet':    testdata.get_ascii_words(280)
                  }
    kinesis.put_record(StreamName = "test_stream",
                       Data = json.dumps(test_tweet),
                       PartitionKey = "partitionkey"
                      )

Overwriting producer.py


Then, we can define a script for our Consumer instance that gets the latest tweet out of the stream, one at a time. After processing each tweet, we then print out the average unique word count per processed tweet as a running average, before jumping on to the next indexed tweet in our Kinesis stream shard to do the same thing for as long as our program is running.

In [5]:
%%file consumer.py

import boto3
import time
import json

kinesis = boto3.client('kinesis', region_name='us-east-1')

shard_it = kinesis.get_shard_iterator(StreamName = "test_stream",
                                     ShardId = 'shardId-000000000000',
                                     ShardIteratorType = 'LATEST'
                                     )["ShardIterator"]

i = 0
s = 0
while 1==1:
    out = kinesis.get_records(ShardIterator = shard_it,
                              Limit = 1
                             )
    for o in out['Records']:
        jdat = json.loads(o['Data'])
        s = s + len(set(jdat['tweet'].split()))
        i = i + 1

    if i != 0:
        print("Average Unique Word Count Per Tweet: " + str(s/i))
        print("Sample of Current Tweet: " + jdat['tweet'][:20])
        print("\n")
        
    shard_it = out['NextShardIterator']
    time.sleep(0.2)

Overwriting consumer.py


As our final preparation step, we'll grab all of the public DNS names of our instances (web addresses that you normally copy from the GUI console to manually ssh into  and record the names of our code files, so that we can easily ssh/scp into the instances and pass them our Python scripts to run.

In [6]:
instance_dns = [instance.public_dns_name 
                 for instance in ec2.instances.all() 
                 if instance.state['Name'] == 'running'
               ]

code = ['producer.py', 'consumer.py']

To copy our files over to our instances and programmatically run commands on them, we can use Python's `scp` and `paramiko` packages. You'll need to install these via `pip install paramiko scp` if you have not already done so.

In [7]:
! pip install paramiko scp

Once we have `scp` and `paramiko` installed, we can copy our producer and consumer Python scripts over to the EC2 instances (designating our first EC2 instance in `instance_dns` as the producer and second EC2 instance as the consumer instance).

Note that, on each instance, we install `boto3` (so that we can access Kinesis through our scripts) and then copy our producer/consumer Python code over to our producer/consumer EC2 instance via `scp`. After we've done this, we install the `testdata` package on the producer instance (which it needs in order to create fake tweets) and instruct it to run our Python producer script. This will write tweets into our Kinesis stream until we stop the script and terminate the producer EC2 instance.

We could also instruct our consumer to get tweets from the stream immediately after this command and this would automatically collect and process the tweets according to the consumer.py script. For the purposes of this demonstration, though, we'll manually ssh into that instance and run the code from the terminal so that we can see the real-time consumption a bit more easily.

In [7]:
import paramiko
from scp import SCPClient
ssh_producer, ssh_consumer = paramiko.SSHClient(), paramiko.SSHClient()

# Initialization of SSH tunnels takes a bit of time; otherwise get connection error on first attempt
time.sleep(5)

# Install boto3 on each EC2 instance and Copy our producer/consumer code onto producer/consumer EC2 instances
instance = 0
stdin, stdout, stderr = [[None, None] for i in range(3)]
for ssh in [ssh_producer, ssh_consumer]:
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(instance_dns[instance],
                username = 'ec2-user',
                key_filename='/home/jclindaniel/MACS_30123.pem')
    
    with SCPClient(ssh.get_transport()) as scp:
        scp.put(code[instance])
    
    if instance == 0:
        stdin[instance], stdout[instance], stderr[instance] = \
            ssh.exec_command("sudo pip install boto3 testdata")
    else:
        stdin[instance], stdout[instance], stderr[instance] = \
            ssh.exec_command("sudo pip install boto3")

    instance += 1

# Block until Producer has installed boto3 and testdata, then start running Producer script:
producer_exit_status = stdout[0].channel.recv_exit_status() 
if producer_exit_status == 0:
    ssh_producer.exec_command("python %s" % code[0])
    print("Producer Instance is Running producer.py\n.........................................")
else:
    print("Error", producer_exit_status)

# Close ssh and show connection instructions for manual access to Consumer Instance
ssh_consumer.close; ssh_producer.close()

print("Connect to Consumer Instance by running: ssh -i \"MACS_30123.pem\" ec2-user@%s" % instance_dns[1])

Producer Instance is Running producer.py
.........................................
Connect to Consumer Instance by running: ssh -i "MACS_30123.pem" ec2-user@ec2-52-207-178-178.compute-1.amazonaws.com


If you run the command above (with the correct path to your actual `.pem` file), you should be inside your Consumer EC2 instance. If you run `python consumer.py`, you should also see a real-time count of the average number of unique words per tweet (along with a sample of the text in the most recent tweet), as in the screenshot:

![](consumer_feed.png)

Cool! Now we can scale this basic architecture up to perform any number of real-time data analyses, if we so desire. Also, if we execute our consumer code remotely via paramiko as well, the process will be entirely remote, so we don't need to keep any local resources running in order to keep streaming/processing real-time data.

As a final note, when you are finished observing the real-time feed from your consumer instance, **be sure to terminate your EC2 instances and delete your Kinesis stream**. You don't want to be paying for these to run continuously! You can do so programmatically by running the following `boto3` code:

In [8]:
# Terminate EC2 Instances:
ec2_client.terminate_instances(InstanceIds=[instance.id for instance in instances])

# Confirm that EC2 instances were terminated:
waiter = ec2_client.get_waiter('instance_terminated')
waiter.wait(InstanceIds=[instance.id for instance in instances])
print("EC2 Instances Successfully Terminated")

# Delete Kinesis Stream (if it currently exists):
try:
    response = kinesis.delete_stream(StreamName='test_stream')
except kinesis.exceptions.ResourceNotFoundException:
    pass

# Confirm that Kinesis Stream was deleted:
waiter = kinesis.get_waiter('stream_not_exists')
waiter.wait(StreamName='test_stream')
print("Kinesis Stream Successfully Deleted")

EC2 Instances Successfully Terminated
Kinesis Stream Successfully Deleted
